# SQL function creation

This notebook stores the code that creates the SQL functions that are then used by the API code. Needs to be run only once (and re-run if any updates of course!).

**Important**:In order to keep the list clean, please prefix any new functions with `api_`

In [2]:
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read("../../config/config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

### Getting catchment area statistics for a given catchment area (by ID)
We'll use precomputed tables, too, as it makes it faster to get accessibility statistics afterwards.

In [3]:
create_sql = """

DROP TABLE IF EXISTS catchment_stats;

CREATE TABLE public.catchment_stats
(
    catchmentid bigint,    
    categorytype text,
    groupname text,    
    population double precision,
    id bigserial,
    CONSTRAINT catchment_stats_id PRIMARY KEY (id),
    CONSTRAINT catchment_stats_unique_key UNIQUE (catchmentid, categorytype, groupname)
);

CREATE INDEX IF NOT EXISTS catchment_stats_index ON public.catchment_stats (catchmentid, categorytype);
"""

execute_sql(create_sql)

Populating the table

In [ ]:
SQL = """
WITH all_h3_ids as (
	SELECT 		
		h3demographics.categorytype,
		h3demographics.h3id, 
		h3demographics.groupname, 
		h3demographics.population 
	FROM h3demographics
),

catchmenth3_ids AS (
	SELECT 
		catchments.catchmentid,
		catchmenth3map.h3id
	FROM catchments
	JOIN catchmenth3map ON catchmenth3map.catchmentid = catchments.catchmentid
)

INSERT INTO catchment_stats (catchmentid, categorytype, groupname, population)
	SELECT c.catchmentid, h.categorytype, h.groupname, SUM(h.population)
		FROM all_h3_ids as h
		JOIN catchmenth3_ids as c ON c.h3id = h.h3id
		GROUP BY c.catchmentid, h.categorytype, h.groupname
"""

execute_sql(SQL)

Defining the function

In [6]:
SQL = """
-- Demographics for a catchment area

CREATE OR REPLACE FUNCTION api_get_demographics_for_catchment(    
    in_categorytype character,
    in_catchment_id integer
)
    RETURNS TABLE
            (                
                groupname   text,
                population   float
            )
    LANGUAGE plpgsql
AS
$demographicsforarea$
BEGIN
    
    RETURN QUERY
        
        SELECT catchment_stats.groupname, catchment_stats.population 
            FROM catchment_stats
            WHERE categorytype = in_categorytype AND catchmentid = in_catchment_id;

END;
$demographicsforarea$;

-- SELECT * FROM api_get_demographics_for_catchment('Race', 1); 
"""

execute_sql(SQL)